## 📚 שלב 1: ייבוא ספריות והכנת כלים

**חדש כאן:**
- `onnxruntime` - להרצת מודל ה-AI (YOLO)
- `pytesseract` - לקריאת טקסט מתמונות

**קבצים שנצטרך:**
- `yolov8n-license_plate.onnx` - מודל ה-AI המאומן
- `/usr/bin/tesseract` - תוכנת ה-OCR

In [1]:
import cv2
import time
import numpy as np
import pytesseract
import onnxruntime as onnxr
import re

from helper_function import (
    run_onnx_inference,  # מריץ את ה-AI
    decode_yolov8,       # מפענח את התוצאות
    free_camera          # משחרר מצלמה תקועה
)

# הגדרות Tesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# הגדרות OCR - מה הוא יחפש
CONFIG_TESS = (
    "--oem 1 --psm 7 "  # מצב זיהוי
    "-c tessedit_char_whitelist=0123456789 "  # רק ספרות!
    "-c classify_bln_numeric_mode=1 "
    "-c user_defined_dpi=200 "
)

print("✅ ספריות נטענו!")
print(f"📍 Tesseract: {pytesseract.pytesseract.tesseract_cmd}")

2026-01-19 00:08:44.793849064 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card1/device/vendor"


[INFO] Camera intrinsics K:
[[450.   0. 320.]
 [  0. 600. 240.]
 [  0.   0.   1.]]
[INFO] YOLO model input size: 512x512
✅ ספריות נטענו!
📍 Tesseract: /usr/bin/tesseract


---

## 🤖 שלב 2: טעינת מודל ה-AI (YOLO)

**מה זה YOLO?**  
זה מודל AI שמאומן לזהות לוחיות רכב. הוא "מסומן" איפה הלוחית בתמונה.

**פורמט ONNX:**  
זה פורמט "סטנדרטי" למודלי AI שעובד טוב על Raspberry Pi.

**פרמטרים:**
- `CONF_THRES = 0.2` - סף ביטחון (20% = מקל, 80% = קפדני)
- `IOU_THRES = 0.2` - למניעת זיהויים כפולים
- `INFER_PERIOD_S = 0.03` - כמה פעמים בשנייה להריץ את ה-AI (כל 30ms)

In [2]:
# נתיב למודל
ONNX_PATH = "/home/pi/Yahboom_project/uveye/code/raspberry-plate-recognition/yolov8n-license_plate.onnx"

# פרמטרי זיהוי
CONF_THRES = 0.2      # סף ביטחון - נמוך = מזהה יותר אבל יותר טעויות
IOU_THRES = 0.2       # טיפול בזיהויים חופפים
INFER_PERIOD_S = 0.03 # תדירות הרצת AI (33 פעמים בשנייה)

# טעינת המודל
onnxr.set_default_logger_severity(3)  # השתקת הודעות מיותרות
session = onnxr.InferenceSession(ONNX_PATH, providers=["CPUExecutionProvider"])

# בדיקת קלט המודל
inp = session.get_inputs()[0]
print(f"✅ מודל YOLO נטען!")
print(f"📊 קלט: {inp.name}, צורה: {inp.shape}")
print(f"⚙️ CONF_THRES={CONF_THRES}, IOU_THRES={IOU_THRES}")

✅ מודל YOLO נטען!
📊 קלט: images, צורה: [1, 3, 512, 512]
⚙️ CONF_THRES=0.2, IOU_THRES=0.2


---

## 🔍 שלב 3: זיהוי לוחית פשוט (ללא OCR)

נתחיל רק עם הזיהוי - נראה איפה המודל מוצא לוחיות.

**איך זה עובד:**
1. קריאת תמונה מהמצלמה
2. שליחת התמונה למודל YOLO
3. המודל מחזיר: "מצאתי לוחית ב-X, Y עם רוחב W וגובה H"
4. נצייר ריבוע ירוק סביב הלוחית

**טיפ:** נסו עם תמונת לוחית על מסך הטלפון או הדפסה.

In [6]:
# שחרור מצלמה אם תקועה
free_camera()

import cv2
import time
import ipywidgets as widgets
from IPython.display import display

print("🔍 מזהה לוחיות (ללא OCR)...")
print("הראו לוחית רכב (אפשר על מסך/נייר)")
print("עצרו ע\"י Interrupt של התא (Kernel -> Interrupt)\n")

# --- camera open (use V4L2 on Raspberry Pi if needed) ---
cap = cv2.VideoCapture(0)  # try: cv2.VideoCapture(0, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # may be ignored, but helps when supported

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

# warm-up + drop stale frames
for _ in range(20):
    cap.read()

# --- widget output ---
img = widgets.Image(format="jpeg")
display(img)

# --- streaming loop parameters ---
TARGET_FPS = 20
JPEG_QUALITY = 75
DROP_GRABS = 2
DT = 1.0 / TARGET_FPS

last_infer = 0.0
detections = []

try:
    t_next = time.time()
    while True:
        # drop a couple frames so we always show the newest
        for _ in range(DROP_GRABS):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            print("failed read")
            time.sleep(0.05)
            continue

        now = time.time()

        # הרצת AI רק כל INFER_PERIOD_S (לחסוך CPU)
        if now - last_infer >= INFER_PERIOD_S:
            output = run_onnx_inference(frame)
            detections = decode_yolov8(
                output, frame.shape[1], frame.shape[0],
                CONF_THRES, IOU_THRES
            )
            last_infer = now

        vis = frame.copy()

        # ציור כל הלוחיות שנמצאו
        for det in detections:
            x1, y1, x2, y2 = det["bbox"]
            conf = det["confidence"]

            cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(vis, f"{conf:.2f}", (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # מונה
        cv2.putText(vis, f"Plates: {len(detections)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

        # encode BGR -> JPEG (fast + smooth in notebooks)
        ok, jpg = cv2.imencode(".jpg", vis, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if ok:
            img.value = jpg.tobytes()

        # pace output (prevents flicker + CPU spikes)
        sleep = t_next - time.time()
        if sleep > 0:
            time.sleep(sleep)
        t_next = max(t_next + DT, time.time())

finally:
    cap.release()
    print("✅ סיימתי")


[INFO] Checking for processes using /dev/video0...
[INFO] /dev/video0 is free.
🔍 מזהה לוחיות (ללא OCR)...
הראו לוחית רכב (אפשר על מסך/נייר)
עצרו ע"י Interrupt של התא (Kernel -> Interrupt)



Image(value=b'', format='jpeg')

✅ סיימתי


KeyboardInterrupt: 

---

## 🔤 שלב 4: הוספת OCR - קריאת המספרים

עכשיו נוסיף את השלב השני: קריאת הטקסט מהלוחית.

### תהליך עיבוד התמונה (לשיפור הקריאה):

1. **גזירה (crop)** - חותכים רק את הלוחית + שוליים
2. **המרה לשחור-לבן (grayscale)**
3. **CLAHE** - שיפור ניגודיות (כמו "בהירות אוטומטית")
4. **הגדלה (resize)** - x2.4 - OCR עובד טוב יותר על תמונות גדולות
5. **טשטוש קל (blur)** - מסיר רעש
6. **סף (threshold)** - שחור-לבן חד (ללא אפור)
7. **OCR** - פעמיים: על הרגיל ועל ההפוך (שחור↔לבן)

### פורמט לוחית ישראלית:
- 7 ספרות: `12-345-67`
- 8 ספרות: `123-45-678`

In [7]:
# הגדרות OCR
OCR_EVERY_N_FRAMES = 3  # OCR רק כל 3 פריימים (חוסך זמן)
MARGIN_X = 0.10         # שוליים אופקיים (10%)
MARGIN_Y = 0.25         # שוליים אנכיים (25%)

# CLAHE - לשיפור ניגודיות
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

def normalize_plate(raw_text):
    """ממיר OCR גולמי לפורמט לוחית"""
    # הסרת כל מה שלא ספרה
    digits = re.sub(r"\D", "", raw_text or "")
    
    if len(digits) == 7:
        return f"{digits[:2]}-{digits[2:5]}-{digits[5:]}"
    elif len(digits) == 8:
        return f"{digits[:3]}-{digits[3:5]}-{digits[5:]}"
    return None

def crop_plate(frame, bbox):
    """גוזר את הלוחית עם שוליים"""
    x1, y1, x2, y2 = bbox
    w, h = x2 - x1, y2 - y1
    mx = int(w * MARGIN_X)
    my = int(h * MARGIN_Y)
    
    return frame[
        max(0, y1 - my):min(frame.shape[0], y2 + my),
        max(0, x1 - mx):min(frame.shape[1], x2 + mx)
    ]

def ocr_plate(crop_img):
    """מבצע OCR על תמונת לוחית"""
    if crop_img is None or crop_img.size == 0:
        return None
    
    # עיבוד
    gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    gray = clahe.apply(gray)
    gray = cv2.resize(gray, None, fx=2.4, fy=2.4, interpolation=cv2.INTER_CUBIC)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # ניסיון OCR על שתי גרסאות (רגיל + הפוך)
    for img in (binary, 255 - binary):
        text = pytesseract.image_to_string(img, config=CONFIG_TESS)
        plate = normalize_plate(text)
        if plate:
            return plate
    
    return None

print("✅ פונקציות OCR מוכנות!")
print(f"📋 פורמטים: XX-XXX-XX (7) או XXX-XX-XXX (8)")

✅ פונקציות OCR מוכנות!
📋 פורמטים: XX-XXX-XX (7) או XXX-XX-XXX (8)


---

## 🎯 שלב 5: זיהוי + קריאה מלאה

עכשיו נשלב הכל: זיהוי + OCR.

**מה יוצג:**
- ריבוע ירוק סביב הלוחית
- הטקסט שנקרא (או "----" אם לא הצליח)
- צבע ירוק אם הצליח לקרוא פורמט חוקי

In [8]:
free_camera()

import cv2
import time
import ipywidgets as widgets
from IPython.display import display

print("🔍 זיהוי + קריאה מלאה")
print("הראו לוחית רכב - המערכת תקרא את המספר")
print("עצרו ע\"י Interrupt של התא (Kernel -> Interrupt)\n")

# --- Best-practice smooth live camera in Jupyter: ipywidgets.Image (no matplotlib redraw)
# --- camera open (use V4L2 on Raspberry Pi if needed) ---
cap = cv2.VideoCapture(0)  # try: cv2.VideoCapture(0, cv2.CAP_V4L2)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # may be ignored, but helps when supported

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

# warm-up + drop stale frames
for _ in range(20):
    cap.read()

# --- widget output ---
img = widgets.Image(format="jpeg")
display(img)

# --- streaming loop parameters ---
TARGET_FPS = 20
JPEG_QUALITY = 75
DROP_GRABS = 2
DT = 1.0 / TARGET_FPS

last_infer = 0.0
detections = []
frame_count = 0
last_ocr_result = {}

try:
    t_next = time.time()
    while True:
        # drop a couple frames so we always show the newest
        for _ in range(DROP_GRABS):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            print("failed read")
            time.sleep(0.05)
            continue

        now = time.time()
        frame_count += 1

        # זיהוי
        if now - last_infer >= INFER_PERIOD_S:
            output = run_onnx_inference(frame)
            detections = decode_yolov8(
                output, frame.shape[1], frame.shape[0],
                CONF_THRES, IOU_THRES
            )
            last_infer = now

        vis = frame.copy()

        if detections:
            # נעבוד על הלוחית הכי בטוחה
            best = max(detections, key=lambda d: d["confidence"])
            x1, y1, x2, y2 = best["bbox"]

            cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # OCR כל N פריימים
            if frame_count % OCR_EVERY_N_FRAMES == 0:
                crop = crop_plate(frame, (x1, y1, x2, y2))
                plate_text = ocr_plate(crop)

                if plate_text:
                    last_ocr_result = {
                        "text": plate_text,
                        "time": now,
                        "bbox": (x1, y1, x2, y2)
                    }
                    print(f"📋 קראתי: {plate_text}")

            # הצגת תוצאה אחרונה (אם קיימת ועדכנית)
            if last_ocr_result and (now - last_ocr_result.get("time", 0)) < 2.0:
                txt = last_ocr_result["text"]
                cv2.putText(vis, f"Plate: {txt}", (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
            else:
                cv2.putText(vis, "OCR: ----", (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 3)

        # show in notebook
        ok, jpg = cv2.imencode(".jpg", vis, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if ok:
            img.value = jpg.tobytes()

        # pace output
        sleep = t_next - time.time()
        if sleep > 0:
            time.sleep(sleep)
        now2 = time.time()
        t_next = max(t_next + DT, now2)

finally:
    cap.release()
    print("✅ סיימתי")


[INFO] Checking for processes using /dev/video0...
[INFO] /dev/video0 is free.
🔍 זיהוי + קריאה מלאה
הראו לוחית רכב - המערכת תקרא את המספר
עצרו ע"י Interrupt של התא (Kernel -> Interrupt)



Image(value=b'', format='jpeg')

📋 קראתי: 911-79-181
📋 קראתי: 911-79-181
📋 קראתי: 91-179-48
✅ סיימתי


KeyboardInterrupt: 

---

## 🎯 שלב 6: חיפוש לוחית ספציפית

עכשיו נוסיף "משימה": למצוא לוחית מסוימת!

**איך זה עובד:**
1. הגדרנו לוחית מטרה: `TARGET_PLATE = "91-179-18"`
2. המערכת מזהה + קוראת לוחיות
3. כשמוצאת את המטרה → **LOCK!** (לא מחפשת יותר)
4. הצגה ירוקה + הודעה

**שימוש:** משחק "מצא את הרכב" או בקרת כניסה לחניון.

In [9]:
# לוחית מטרה (שנו לפי הצורך!)
TARGET_PLATE = "91-179-18"

free_camera()

import cv2
import time
import ipywidgets as widgets
from IPython.display import display

print(f"🎯 מחפש לוחית: {TARGET_PLATE}")
print("הראו לוחיות שונות עד שתמצא את המטרה")
print("עצרו ע\"י Interrupt של התא (Kernel -> Interrupt)\n")

# --- Best-practice smooth live camera in Jupyter: ipywidgets.Image (no matplotlib redraw)
# --- camera open (use V4L2 on Raspberry Pi if needed) ---
cap = cv2.VideoCapture(0)  # try: cv2.VideoCapture(0, cv2.CAP_V4L2)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # may be ignored, but helps when supported

if not cap.isOpened():
    raise RuntimeError("Cannot open camera")

# warm-up + drop stale frames
for _ in range(20):
    cap.read()

# --- widget output ---
img = widgets.Image(format="jpeg")
display(img)

# --- streaming loop parameters ---
TARGET_FPS = 20
JPEG_QUALITY = 75
DROP_GRABS = 2
DT = 1.0 / TARGET_FPS

last_infer = 0.0
detections = []
frame_count = 0
target_found = False
target_data = None
last_guess = None

try:
    t_next = time.time()
    while True:
        # drop a couple frames so we always show the newest
        for _ in range(DROP_GRABS):
            cap.grab()

        ret, frame = cap.read()
        if not ret:
            print("failed read")
            time.sleep(0.05)
            continue

        now = time.time()
        frame_count += 1

        # זיהוי
        if now - last_infer >= INFER_PERIOD_S:
            output = run_onnx_inference(frame)
            detections = decode_yolov8(
                output, frame.shape[1], frame.shape[0],
                CONF_THRES, IOU_THRES
            )
            last_infer = now

        vis = frame.copy()

        if detections:
            best = max(detections, key=lambda d: d["confidence"])
            x1, y1, x2, y2 = best["bbox"]

            # OCR רק אם עדיין לא מצאנו
            if not target_found and frame_count % OCR_EVERY_N_FRAMES == 0:
                crop = crop_plate(frame, (x1, y1, x2, y2))
                plate_text = ocr_plate(crop)

                if plate_text:
                    last_guess = plate_text
                    print(f"📋 קראתי: {plate_text}")

                    # בדיקה מול מטרה
                    if plate_text == TARGET_PLATE:
                        target_found = True
                        target_data = {
                            "text": plate_text,
                            "bbox": (x1, y1, x2, y2),
                            "time": now
                        }
                        print(f"\n🎉🎉🎉 מצאתי את המטרה: {TARGET_PLATE} 🎉🎉🎉\n")

            # ציור
            color = (0, 255, 0) if target_found else (255, 255, 0)
            cv2.rectangle(vis, (x1, y1), (x2, y2), color, 3 if target_found else 2)

        # תצוגה
        if target_found:
            cv2.putText(vis, f"TARGET: {TARGET_PLATE}", (10, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            cv2.putText(vis, "*** FOUND ***", (10, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
        else:
            cv2.putText(vis, f"Looking for: {TARGET_PLATE}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            if last_guess:
                cv2.putText(vis, f"Last: {last_guess}", (10, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        # show in notebook
        ok, jpg = cv2.imencode(".jpg", vis, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
        if ok:
            img.value = jpg.tobytes()

        # pace output
        sleep = t_next - time.time()
        if sleep > 0:
            time.sleep(sleep)
        now2 = time.time()
        t_next = max(t_next + DT, now2)

finally:
    cap.release()

if target_found:
    print(f"✅ סיימתי - מצאתי את {TARGET_PLATE}!")
else:
    print("⚠️ סיימתי - לא מצאתי את המטרה")


[INFO] Checking for processes using /dev/video0...
[INFO] /dev/video0 is free.
🎯 מחפש לוחית: 91-179-18
הראו לוחיות שונות עד שתמצא את המטרה
עצרו ע"י Interrupt של התא (Kernel -> Interrupt)



Image(value=b'', format='jpeg')

📋 קראתי: 191-17-918
📋 קראתי: 19-179-18
📋 קראתי: 191-17-918
📋 קראתי: 191-17-918
📋 קראתי: 19-179-81
📋 קראתי: 19-179-18
📋 קראתי: 19-179-81
📋 קראתי: 191-17-918
📋 קראתי: 91-179-18

🎉🎉🎉 מצאתי את המטרה: 91-179-18 🎉🎉🎉



KeyboardInterrupt: 

---

## 📊 סיכום - מושגים ופרמטרים

### מושגים:

| מושג | הסבר |
|------|------|
| **YOLO** | מודל AI לזיהוי אובייקטים (כאן: לוחיות) |
| **ONNX** | פורמט למודלי AI שעובד על מכשירים חלשים |
| **OCR** | Optical Character Recognition - קריאת טקסט |
| **Tesseract** | תוכנת OCR בקוד פתוח |
| **CLAHE** | שיפור ניגודיות אדפטיבי |
| **Threshold** | המרה לשחור-לבן חד |
| **Confidence** | רמת ביטחון של ה-AI (0.0-1.0) |
| **IOU** | Intersection Over Union - למניעת זיהויים כפולים |

### פרמטרים לכיוונון:

| פרמטר | טווח | ברירת מחדל | השפעה |
|--------|------|------------|--------|
| `CONF_THRES` | 0.1-0.8 | 0.2 | סף זיהוי - נמוך=יותר זיהויים (ויותר טעויות) |
| `IOU_THRES` | 0.1-0.5 | 0.2 | מניעת כפילויות |
| `INFER_PERIOD_S` | 0.03-0.2 | 0.03 | תדירות AI - נמוך=מדויק אבל איטי |
| `OCR_EVERY_N_FRAMES` | 1-10 | 3 | כל כמה פריימים OCR - גבוה=מהיר אבל פחות רספונסיבי |
| `MARGIN_X/Y` | 0.0-0.3 | 0.1/0.25 | שוליים לגזירה - גדול=יותר הקשר |

### שרשרת עיבוד:

```
תמונה מקורית
    ↓
YOLO (זיהוי bbox)
    ↓
Crop + Margins (גזירה)
    ↓
Grayscale (שחור-לבן)
    ↓
CLAHE (שיפור ניגודיות)
    ↓
Resize x2.4 (הגדלה)
    ↓
Blur (טשטוש קל)
    ↓
Threshold (סף)
    ↓
Tesseract OCR
    ↓
Normalize (פורמט)
```

---

## 💡 רעיונות לשיפורים

1. **מסד נתונים:**
   - שמרו כל לוחית שנקראה עם timestamp
   - בסוף: "ראיתי 5 לוחיות שונות"

2. **סטטיסטיקות:**
   - כמה זיהויים? כמה מהם הצליחו OCR?
   - אחוז הצלחה

3. **משחק "זיכרון":**
   - הראו 3 לוחיות
   - המערכת תזכור אותן
   - הראו שוב - האם זוכרת?

4. **בקרת כניסה:**
   - רשימת לוחיות מורשות
   - אם נמצאה → "כניסה מאושרת"
   - אחרת → "כניסה נדחתה"

**בהצלחה! 🚗🔍**